# Cow Teat Image Classification

This notebook implements a custom CNN to classify cow teat images using PyTorch. 
It includes improvements such as Batch Normalization, Dropout, and specific data loading logic for the project's split dataset.

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, ConcatDataset

# ============ CONFIG ============
BASE_DIR = os.getcwd()
TRAIN_DIRS = [
    os.path.join(BASE_DIR, "Train_1", "Score_1"), # Class 0
    os.path.join(BASE_DIR, "Train_1", "Score_2"), # Class 1
    os.path.join(BASE_DIR, "Train_2", "Score_3"), # Class 2
    os.path.join(BASE_DIR, "Train_2", "Score_4"), # Class 3
]
TEST_DIR = os.path.join(BASE_DIR, "Test-2")
OUTPUT_DIR = os.path.join(BASE_DIR, "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

BATCH_SIZE = 32
NUM_EPOCHS = 30
LEARNING_RATE = 0.001
DEVICE = torch.device('mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu'))
print(f"Using device: {DEVICE}")

In [ ]:
# ============ DATASET CLASSES ============

class SingleClassDataset(Dataset):
    """Dataset for a single folder representing one class."""
    def __init__(self, root_dir, class_label, transform=None):
        self.root_dir = root_dir
        self.class_label = class_label
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            image = Image.new('RGB', (224, 224))
            
        if self.transform:
            image = self.transform(image)
            
        return image, self.class_label

class CheckTestDataset(Dataset):
    """Test dataset that parses class from filename (e.g. ..._C1_...)."""
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = sorted([f for f in os.listdir(root_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        # Parse label: Look for _C1_, _C2_, etc.
        match = re.search(r'_C(\d)_', img_name)
        if match:
            class_num = int(match.group(1))
            label = class_num - 1 # 1-based to 0-based
        else:
            # print(f"Warning: Could not parse class from {img_name}, defaulting to 0")
            label = 0
            
        return image, label, img_name

In [ ]:
# ============ TRANSFORMS ============

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ============ DATA LOADING ============

train_datasets = []
for i, directory in enumerate(TRAIN_DIRS):
    if os.path.exists(directory):
        print(f"Loading Class {i} from {directory}")
        ds = SingleClassDataset(directory, class_label=i, transform=train_transform)
        train_datasets.append(ds)
        print(f"  Found {len(ds)} images.")
    else:
        print(f"WARNING: Directory not found: {directory}")

if not train_datasets:
    print("No training data found! Check paths.")
else:
    full_train_dataset = ConcatDataset(train_datasets)
    # Set num_workers=0 for macOS compatibility
    train_loader = DataLoader(full_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

if os.path.exists(TEST_DIR):
    test_dataset = CheckTestDataset(TEST_DIR, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    print(f"Loading Test data from {TEST_DIR}")
    print(f"  Found {len(test_dataset)} images.")
else:
    print(f"WARNING: Test directory not found: {TEST_DIR}")
    test_loader = None

In [ ]:
# ============ MODEL ============

class ImprovedNet(nn.Module):
    def __init__(self):
        super(ImprovedNet, self).__init__()
        # 1. Conv Block
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        # 2. Conv Block
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # 3. Conv Block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # 4. Conv Block
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        # 5. Conv Block
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Fully Connected
        self.fc1 = nn.Linear(512 * 7 * 7, 1024)
        self.drop1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(1024, 512)
        self.drop2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(512, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x)))) # 112
        x = self.pool(F.relu(self.bn2(self.conv2(x)))) # 56
        x = self.pool(F.relu(self.bn3(self.conv3(x)))) # 28
        x = self.pool(F.relu(self.bn4(self.conv4(x)))) # 14
        x = self.pool(F.relu(self.bn5(self.conv5(x)))) # 7
        
        x = x.view(-1, 512 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        x = self.fc3(x)
        return x

In [ ]:
# ============ TRAINING ============

model = ImprovedNet().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

print("Starting training...")
loss_values = []

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    if train_loader:
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        scheduler.step()
        epoch_loss = running_loss / len(train_loader)
        loss_values.append(epoch_loss)
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {epoch_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

print("Training complete.")

In [ ]:
# Plot Loss
plt.figure(figsize=(8, 5))
plt.plot(loss_values, 'b-o', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss per Epoch')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(OUTPUT_DIR, 'training_loss.png'))
plt.show()

In [ ]:
# ============ EVALUATION ============

if test_loader:
    print("Evaluating on Test Set...")
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels, filenames in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    
    # Save Model
    torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, 'model.pt'))
    print(f"Model saved to {os.path.join(OUTPUT_DIR, 'model.pt')}")
    
    # Predictions CSV
    test_files = test_dataset.image_files
    df = pd.DataFrame({'Image': test_files, 'True Label': all_labels, 'Predicted': all_preds})
    df.to_csv(os.path.join(OUTPUT_DIR, 'predictions.csv'), index=False)
    print("Predictions saved to outputs/predictions.csv")

## Links
- [Project Paper](./Image%20classification%20using%20CNN%20with%20pytorch.pdf)